In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
import time
import os


In [2]:
# Unzip the dataset
zip_path = 'reduced_iam_handwritten.txt.zip'
data_dir = 'reduced_iam_handwritten'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(data_dir)


NameError: name 'zipfile' is not defined

In [3]:
# Function to read and preprocess text data
def preprocess_data(data_dir):
    text_sequences = []
    for root, _, files in os.walk(data_dir):
        for file in files:
            if file.endswith(".txt"):
                with open(os.path.join(root, file), 'r') as f:
                    text = f.read().strip()
                    text_sequences.append(text)
    return text_sequences


In [4]:
# Load and preprocess the data
text_sequences = preprocess_data(data_dir)

In [6]:
# Define vocabulary and mappings
vocab = sorted(set(''.join(text_sequences)))
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
    return np.array([char2idx[c] for c in text])


In [7]:
# Convert text to integer sequences and pad them
input_sequences = [text_to_int(seq) for seq in text_sequences]
input_sequences = pad_sequences(input_sequences, padding='post')

x_train = input_sequences[:, :-1]
y_train = input_sequences[:, 1:]

In [8]:
# Define the model
vocab_size = len(vocab)
embedding_dim = 64
rnn_units = 256

model = tf.keras.Sequential([
    Embedding(vocab_size, embedding_dim),
    SimpleRNN(rnn_units, return_sequences=True),
    Dense(vocab_size, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn (SimpleRNN)               │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [9]:
# Reduce the batch size to speed up the training time per epoch
batch_size = 16